In [32]:
import pickle

f_name = "../EXP_id03.pkl"
with open(f_name, "rb") as f:
    _results = pickle.load(f)

In [33]:
import pandas as pd

N_AGENTS = len(_results) + 1
results = pd.DataFrame(0.0, index=range(N_AGENTS), columns=range(N_AGENTS))
results

,0,1,2,3,4,5,6,7,8,9,...,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
for i, _res_i in enumerate(_results):
    for j, _res_ij in _res_i.items():
        results.loc[i + 1, j] += _res_ij["wins"]
        results.loc[j, i + 1] += _res_ij["losses"]

        # draws are half win and half loss
        results.loc[i + 1, j] += _res_ij["draws"] / 2
        results.loc[j, i + 1] += _res_ij["draws"] / 2

# Access wins of i over j
w = lambda i, j: results[j][i]

In [35]:
w(3, 1)  # wins of 1 over 2

10.0

In [36]:
results.iloc[3, 1]

10.0

In [37]:
results[3][2]

0.0

In [135]:
from math import prod


def calculate_BradleyTerry(
    score: dict[int, float],
    W: pd.DataFrame,
    EPOCHS: int = 4,
    diff_threshold: float = 1e-3,
    verbose=False,
):
    """Calculate Bradley-Terry scores given initial scores and a win matrix W.
    # Arguments:
        score: (dict) Initial scores for each agent.
        W: A DataFrame where W.iloc[i, j] is the number of wins of agent i over agent j.
        EPOCHS: Maximum number of iterations to perform.
        diff_threshold: Threshold for convergence based on the change in scores.
        verbose: If True, print progress information.
    """
    N_AGENTS = len(score)
    for _ in range(EPOCHS):
        # score_new = [0.0] * N_AGENTS
        score_old = score.copy()
        for i in score:
            # for i in score.__reversed__(): # cambio de escala...
            _num, _den = 0, 0
            for j in score:
                if i == j:
                    continue
                wins_of_i_over_j = float(W.loc[i, j])  # type: ignore
                loses_of_i_to_j = float(W.loc[j, i])  # type: ignore

                _num += wins_of_i_over_j * score[j] / (score[i] + score[j])
                _den += loses_of_i_to_j / (score[i] + score[j])

            score[i] = _num / _den
            # score_new[i] = _num / _den

            # Normalize by geometric mean -- not used for numerical stability
            # _m_g = prod([x ** (1 / N_AGENTS) for x in score_new])
            # score_new = [x / _m_g for x in score_new]
        diff_norm2 = (
            sum((score[i] - score_old[i]) ** 2 for i in score) ** 0.5 / N_AGENTS
        )
        # if diff_norm2 < diff_threshold:
        #     break
        if verbose:
            print(f"Diff norm2: {diff_norm2:.6f}")
            print([f"{x:.3f}" for x in score])
    return score

In [68]:
results.loc[-1, 0] = 3
results

,-1,0,1,2,3,4,5,6,7,8,...,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099
-1,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.0,0.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,5.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,5.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
results[1] = 0
results

,-1,0,1,2,3,4,5,6,7,8,...,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099
-1,0.0,3.0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.0,0.0,0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,5.0,0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,5.0,0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,5.0,0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1097,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1098,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
results.loc[2] = 0
results[2] = 0
results

,-1,0,1,2,3,4,5,6,7,8,...,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099
-1,0.0,3.0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.0,0.0,0,0,5.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,5.0,0,0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,5.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1097,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1098,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
v = [
    -1,
] + list(range(1100))
v

[-1,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 

In [138]:
# agent: -1 dummy player, everybody wins and loses against it
# agent 0 is the random agent at the start of training
scores = {-1: 1.0, 0: 1.0}  # initial scores
results = pd.DataFrame(0.0, index=v, columns=v)
results.loc[0, -1] = 1.0
results.loc[-1, 0] = 1.0

BT_UPDATE_FREQ = 100
NUM_TOTAL = len(_results)
for i_m1, _res in enumerate(_results):
    # add new agent to the pool
    i = i_m1 + 1  # index of the new agent
    if i % 5 == 0:
        print(f"Adding agent {i}")
    scores[i] = 1.0
    # print(i, scores)
    # results.loc[i] = 0.0
    # results[i] = 0.0
    # add results of the new agent
    for j, _res_ij in _res.items():
        # print(_res_ij)
        results.loc[i, j] += _res_ij["wins"]
        results.loc[j, i] += _res_ij["losses"]

        # draws are half win and half loss
        results.loc[i, j] += _res_ij["draws"] / 2
        results.loc[j, i] += _res_ij["draws"] / 2
    results.loc[-1, i] = 1.0
    results.loc[i, -1] = 1.0

    # recalculate scores
    if i % BT_UPDATE_FREQ == 0 or i == NUM_TOTAL:
        print(f"Updating scores at agent {i}")
        scores = calculate_BradleyTerry(scores, results, EPOCHS=2, verbose=False)
        # print(scores)
scores

Adding agent 5
Adding agent 10
Adding agent 15
Adding agent 20
Adding agent 25
Adding agent 30
Adding agent 35
Adding agent 40
Adding agent 45
Adding agent 50
Adding agent 55
Adding agent 60
Adding agent 65
Adding agent 70
Adding agent 75
Adding agent 80
Adding agent 85
Adding agent 90
Adding agent 95
Adding agent 100
Updating scores at agent 100
Adding agent 105
Adding agent 110
Adding agent 115
Adding agent 120
Adding agent 125
Adding agent 130
Adding agent 135
Adding agent 140
Adding agent 145
Adding agent 150
Adding agent 155
Adding agent 160
Adding agent 165
Adding agent 170
Adding agent 175
Adding agent 180
Adding agent 185
Adding agent 190
Adding agent 195
Adding agent 200
Updating scores at agent 200
Adding agent 205
Adding agent 210
Adding agent 215
Adding agent 220
Adding agent 225
Adding agent 230
Adding agent 235
Adding agent 240
Adding agent 245
Adding agent 250
Adding agent 255
Adding agent 260
Adding agent 265
Adding agent 270
Adding agent 275
Adding agent 280
Adding age

{-1: 1.015964070131824,
 0: 1.213782379943184,
 1: 1.13736216622223,
 2: 1.1208453828667384,
 3: 1.2400387412054967,
 4: 1.2864591311755684,
 5: 1.626335898684868,
 6: 1.8396424628365782,
 7: 2.009595129924698,
 8: 2.04623076509478,
 9: 2.430076931239591,
 10: 2.375658210545629,
 11: 2.202867036097389,
 12: 1.7261511703761514,
 13: 1.952903235823435,
 14: 2.211741917298086,
 15: 2.144515034853184,
 16: 2.12761485495893,
 17: 2.1347890745861906,
 18: 2.220076653350216,
 19: 2.1185915444928862,
 20: 1.812248249584749,
 21: 1.6312456209953121,
 22: 1.609737360026223,
 23: 1.921212441715467,
 24: 1.8729047396478165,
 25: 1.6615203692244576,
 26: 1.6479098804532328,
 27: 1.855415322530092,
 28: 1.7217654227921761,
 29: 1.9228001786061464,
 30: 1.9204890863682593,
 31: 1.3992258510003297,
 32: 1.1501743472349069,
 33: 0.9796439675434582,
 34: 1.0963617025849541,
 35: 0.9755627970941412,
 36: 0.8718185503041699,
 37: 0.8204143738933922,
 38: 1.0589040005982981,
 39: 1.1563891166461104,
 40: 1

In [140]:
for i, v in scores.items():
    print(i, v)

-1 1.015964070131824
0 1.213782379943184
1 1.13736216622223
2 1.1208453828667384
3 1.2400387412054967
4 1.2864591311755684
5 1.626335898684868
6 1.8396424628365782
7 2.009595129924698
8 2.04623076509478
9 2.430076931239591
10 2.375658210545629
11 2.202867036097389
12 1.7261511703761514
13 1.952903235823435
14 2.211741917298086
15 2.144515034853184
16 2.12761485495893
17 2.1347890745861906
18 2.220076653350216
19 2.1185915444928862
20 1.812248249584749
21 1.6312456209953121
22 1.609737360026223
23 1.921212441715467
24 1.8729047396478165
25 1.6615203692244576
26 1.6479098804532328
27 1.855415322530092
28 1.7217654227921761
29 1.9228001786061464
30 1.9204890863682593
31 1.3992258510003297
32 1.1501743472349069
33 0.9796439675434582
34 1.0963617025849541
35 0.9755627970941412
36 0.8718185503041699
37 0.8204143738933922
38 1.0589040005982981
39 1.1563891166461104
40 1.2415275083954
41 1.4500688817358056
42 1.6164783120541193
43 1.1766160634892857
44 1.020757232307356
45 1.1328747452230148
4

In [141]:
a = input()